In [1]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np

/home/hoang/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from raw_data import air_quality_train_data

dt = air_quality_train_data("/home/hoang/Documents/CodeSpace/air-quality-forecasting/data/data-train")

In [16]:
feats = []

for key in ("input", "output"):
    for station in dt[key].values():
        feats += station["data"]["temperature"].dropna().tolist()

feats = np.array(feats)

In [17]:
feats.min(), (feats.max() - feats.min())

(8.8, 37.11)

In [13]:
feats.mean(), feats.std()

(45.063545172080076, 42.227863829824535)

In [28]:
L = 24
x = []

for i in range(375):
    _x = []
    for station in dt["input"].values():
        stdt = station["data"]["PM2.5"]
        start_idx = L * i
        end_idx = start_idx + L

        frame = stdt[start_idx : end_idx]

        if frame.isna().sum() == 0:
            _x.append(frame.tolist())

    x.append(torch.tensor(_x))

In [ ]:
x[-1]

In [30]:
class MissingDataset(Dataset):
    def __init__(self, path: str, frame_size: int, usecol: str = "PM2.5"):
        self.data = self._preprocessing(path, usecol, frame_size)

    def _preprocessing(self, path: str, usecol: str, frame_size: int):
        raw = air_quality_train_data(path)
        data_len = int(9000 / frame_size)
        
        x = []
        for i in range(data_len):
            _x = []
            for k in ("input", "output"):
                for station in dt[k].values():
                    stdt = station["data"][usecol]
                    start_idx = frame_size * i
                    end_idx = start_idx + frame_size

                    frame = stdt[start_idx : end_idx]

                    if frame.isna().sum() == 0:
                        _x.append(frame.tolist())

            x.append(torch.tensor(_x))

        return x

    def __getitem__(self, index):
        return x[index]

    def __len__(self):
        return len(x)

In [31]:
dts = MissingDataset(
    "/home/hoang/Documents/CodeSpace/air-quality-forecasting/data/data-train",
    frame_size=24,
)

In [35]:
dts[1].shape

torch.Size([10, 24])